In [1]:
import numpy as np
import matplotlib.pyplot as plt
import databehandling.matlib as matlib
import matplotlib

In [2]:
# Parameters

m = {'name': 'UDGRP', 'units': 'MPa-mm-Mg', 'rho': 2e-09, 
     'E1': 38000, 'E2': 8500, 'v12': 0.28, 'G12': 3400, 
     'XT': 1150, 'YT': 40, "ZT":0, "ZC":0, 'XC': 700, 'YC': 120, 'S12': 60, 'S23': 40, "S13":0, 'f12': -0.5}

d1 = 0.9 # Degradation factor transverse mod and shear mod
d2 = 0.1 # Degradation factor compression
m2 = {'name': 'UDGRP', 'units': 'MPa-mm-Mg', 'rho': 2e-09, 
     'E1': 38000, 'E2': 8500*(1-d1), 'v12': 0.28, 'G12': 3400*(1-d1), 
     'XT': 1150, 'YT': 40, "ZT":0, "ZC":0, 'XC': 700*(1-d2), 'YC': 120, 'S12': 60, 'S23': 40, "S13":0, 'f12': -0.5}

legendsize = 15
def set_plot_props():
    legendsize = 15
    plt.figure(figsize=(9,9))
    #matplotlib.rc('xtick', labelsize=15) 
    #matplotlib.rc('ytick', labelsize=15)
    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=18)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [7]:
# Functions

def S2D(m):
    return np.array([[        1/m['E1'], -m['v12']/m['E1'],          0],
                     [-m['v12']/m['E1'],         1/m['E2'],          0],
                     [                0,                 0, 1/m['G12']]], float)

def Q2D(m):
    S=S2D(m)
    return np.linalg.inv(S)


def T2Ds(a):
    c,s = np.cos(np.radians(a)), np.sin(np.radians(a))
    return np.array([[ c*c ,  s*s ,   2*c*s],
                     [ s*s ,  c*c ,  -2*c*s],
                     [-c*s,   c*s , c*c-s*s]], float)

# Strain transform, global to local
def T2De(a):
    c,s = np.cos(np.radians(a)), np.sin(np.radians(a))
    return np.array([[   c*c,   s*s,     c*s ],
                     [   s*s,   c*c,    -c*s ],
                     [-2*c*s, 2*c*s, c*c-s*s ]], float)

def Q2Dtransform(Q,a):
    return np.dot(np.linalg.inv( T2Ds(a) ) , np.dot(Q,T2De(a)) )

def laminateThickness(layup):
    return sum([layer['thi'] for layer in layup])

def fE_max_stress(s, m):
    return max(s[0]/m["XT"], -s[0]/m["XC"])
    


def computeA(layup):
    A=np.zeros((3,3),float)
    hbot = -laminateThickness(layup)/2        # bottom of first layer
    for layer in layup:
        m = layer['mat']
        Q = Q2D(m)
        a = layer['ori']
        Qt = Q2Dtransform(Q,a)
        htop = hbot + layer['thi']   # top of current layer
        A = A + Qt*(htop-hbot)
        hbot=htop                    # for the next layer
    return A

def fE_hashin(s,m):
    s1,s2,s3,s23,s13,s12=s[0],s[1],s[2],s[3],s[4],s[5]
    XT,YT,ZT,XC,YC,ZC,S12,S13,S23 = m['XT'], m['YT'], m['ZT'], m['XC'], m['YC'], m['ZC'], m['S12'], m['S13'], m['S23']
    if s1>0:
        R = ( 1/( (s1/XT)**2 + (1/S12**2)*(s12**2 + s13**2) ) )**0.5
        fE_FF=1/R
    if s1<=0:
        fE_FF=-s1/XC
    if (s2+s3)>=0:
        temp=( (1/YT**2)*(s2+s3)**2+(1/S23**2)*(s23**2-s2*s3)+(1/S12**2)*(s12**2+s13**2) )
        if temp==0:
            fE_IFF = 0
        else:
            R = (1/temp)**0.5
            fE_IFF = 1/R
    if (s2+s3)<0:
        b = (1/YC)*((YC/(2*S23))**2-1)*(s2+s3)
        a = (1/(4*S23**2))*(s2+s3)**2+(1/S23**2)*(s23**2-s2*s3)+(1/S12**2)*(s12**2+s13**2)
        if a==0:
            fE_IFF = 0.0
        else:
            c=-1
            R=(-b+(b**2-4*a*c)**0.5)/(2*a)
            fE_IFF = 1/R
    return fE_FF, fE_IFF

def fE_hashin_2D(s, m):
    sV = s
    YT = m["YT"]
    S12 = m["S12"]
    YC = m["YC"]
    S23 = m["S23"]
    if(sV[1] >= 0):
        temp = ((1/YT**2)*sV[1]**2)+((1/S12**2)*sV[2]**2)
        if temp == 0:
            fe = 0
        else:
            R = (1/temp)**0.5
            fe = 1/R
    if(sV[1] < 0):
        #solving with abc-formula
        a = (sV[1]**2/(4*S23**2))+(sV[2]**2/S12**2)
        b = (1/YC)*(((YC**2/(4*S23**2))-1)*sV[1])
        c = -1
        if a == 0: #avoid dividing by zero
            fe = 0
        else:
            R = (-b+(b**2-4*a*c)**0.5)/(2*a)
            fe = 1/R
    return 0, fe

def solve_old(layup, R, P, Fx, T):
    t = laminateThickness(layup)
    Nx=P*R/2 + Fx/(2*np.pi*R)
    Ny=P*R
    Ip = 2*np.pi*R**3*t
    Nxy = T*R*t/Ip

    loads=[Nx,Ny,Nxy]
    A = computeA(layup)
    strains_global=np.linalg.solve(A,loads)
    
    fE_ff = 0
    fE_iff = 0
    fE_iff_2D = 0
    stresses = []
    local_strains = []
    Qs = []
    global_strains = []
    global_stresses = []
    results = {}
    
    for layer in layup:
        angle = layer["ori"]
        strain_local = np.matmul(T2De(angle), strains_global)
        mat = layer["mat"]
        Q = Q2D(mat)
        Q_global = Q2Dtransform(Q, angle)
        
        s = np.matmul(Q, strain_local)
        s_whole = [s[0], s[1], 0, 0, 0, s[2]]
        s_global = np.matmul(Q_global, strains_global)
        
        fE_ff = max(fE_ff, fE_max_stress(s, mat))
        fE_iff_2D = max(fE_iff, fE_hashin_2D(s, mat)[1])
        fE_iff = max(fE_iff, fE_hashin(s_whole, mat)[1])

        stresses.append(s)
        local_strains.append(strain_local)
        Qs.append(Q)
        global_strains.append(strains_global)
        global_stresses.append(s_global)
        
        
    results["stresses"] = stresses
    results["fE_ff"] = fE_ff
    results["strains_local"] = local_strains
    results["fE_iff"] = fE_iff
    results["fE_iff_2D"] = fE_iff
    
    results["A"] = A
    results["Q"] = Qs
    results["global_strains"] = global_strains
    results["global_stresses"] =  global_stresses
    results["loads"] = loads
    return results


def solve(layup, R, P, Fx, T):
    t = laminateThickness(layup)
    Nx=P*R/2 + Fx/(2*np.pi*R)
    Ny=P*R
    Ip = 2*np.pi*R**3*t
    Nxy = T*R*t/Ip

    loads=[Nx,Ny,Nxy]
    A = computeA(layup)
    strains_global=np.linalg.solve(A,loads)
    
    fE_ff = 0
    fE_iff = 0
    fE_iff_2D = 0
    stresses = []
    local_strains = []
    Qs = []
    global_strains = []
    global_stresses = []
    results = {}
    
    for layer in layup:
        angle = layer["ori"]
        strain_local = np.matmul(T2De(angle), strains_global)
        mat = layer["mat"]
        Q = Q2D(mat)
        Q_global = Q2Dtransform(Q, angle)
        
        s = np.matmul(Q, strain_local)
        s_whole = [s[0], s[1], 0, 0, 0, s[2]]
        s_global = np.matmul(Q_global, strains_global)
        
        fE_ff = max(fE_ff, fE_max_stress(s, mat))
        #fE_iff_2D = max(fE_iff, fE_hashin_2D(s, mat)[1])
        fE_iff = max(fE_iff, fE_hashin(s_whole, mat)[1])

        stresses.append(s)
        local_strains.append(strain_local)
        Qs.append(Q)
        global_strains.append(strains_global)
        global_stresses.append(s_global)
        
        
    results["stresses"] = stresses
    results["fE_ff"] = fE_ff
    results["strains_local"] = local_strains
    results["fE_iff"] = fE_iff
    results["fE_iff_2D"] = fE_iff
    
    results["A"] = A
    results["Q"] = Qs
    results["global_strains"] = global_strains
    results["global_stresses"] =  global_stresses
    results["loads"] = loads
    
    return results

def loadcase(x):
    p = 0
    Fx = 0
    T = 0
    if x == 1:
        p = 8
    elif x == 2:
        Fx = 100E3
    elif x == 3:
        T = 2E7
    elif x == 4:
        p = 8
        Fx = 100E3
    elif x == 5:
        Fx = 100E3
        T = 2E7
    return p, Fx, T
            
        

In [4]:
# Thickness optimization

P,Fx,T = loadcase(4)
R = 75

set_plot_props()
theta1 = 10
theta2 = 88

theta_B = 47
for i in range(3):
    t = 1
    t1 = np.linspace(0, 1, 100)
    t2 = t - t1
    fE_A = np.zeros_like(t1)
    fE_B = np.zeros_like(t1)

    for i in range(len(t1)):
        layupA =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                     {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                    {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2},
                     {'mat':m,  'ori':  theta2,  'thi':t2[i]/2}]

        layupB =     [{'mat':m,  'ori':  theta_B,  'thi':t/2},
                     {'mat':m,  'ori':  -theta_B,  'thi':t/2}]


        res_iff = solve(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)
        res_iff_B = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)

        #strain_iff = res_iff["global_strains"]
        fE[i] = res_iff["fE_iff"]
        fE_B[i] = res_iff_B["fE_iff"]

    i = fE.argmin() # Optimal fE
    print("Min fE=", fE[i], "t1=", t1[i], "t2=", t2[i])
    print("Min fE_B=", min(fE_B), "t=", t)

    plt.plot(t1, fE)
    plt.plot(t1, fE_B)

NameError: name 'fE' is not defined

<Figure size 648x648 with 0 Axes>

In [ ]:
# Angle optimization for different thicknesses
# Case 1
casenr = 1
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [[10, 45], [10, 50], [10,60], [10,70],[10, 75], [15, 88], [20, 70], [20, 75], [20, 80], [30, 80]]

t = 1
t1 = np.linspace(0, 1, 100)
t2 = t - t1
fE_B = np.zeros_like(t1)
fE_A = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    fE_B = np.zeros_like(t1)
    for i in range(len(t1)):
        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]
        layupB_weakened = [{'mat':m2,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m2,  'ori':  -theta2,  'thi':t2[i]/2}]

        
        fE_B_ff = solve(layupB_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
        fE_B_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_B[i] = max(fE_B_ff,fE_B_iff)
        
    i = fE_B.argmin() # Optimal fE
    label = f"±{theta1}/±{theta2}"
    print("Min fE=", fE_B[i], "t1=", t1[i], "t2=", t2[i])
    plt.plot(t1, fE_B, label=label)
    
    
# Solve for layup A
theta1_A = 54
layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]
layupA_weakened =     [{'mat':m2,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m2,  'ori':  -theta1_A,  'thi':t/2}]


fE_A_iff = solve_old(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
fE_A_ff = solve_old(layupA_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
fE_A = (np.zeros_like(t1)+1)*max(fE_A_iff, fE_A_ff)

label_A = f"±{theta1_A}"
print("min_fE layupA", min(fE_A))
plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
plt.legend()

plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
plt.xlabel("$t_1$/$t$")
plt.ylabel("$f_E$")
plt.ylim([min(fE_A)-2,min(fE_A)*2])
plt.savefig(f'output/Case{casenr} - optimization.png')



In [ ]:
# Angle optimization for different thicknesses - IFF
# Case 2
casenr = 2
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [[10, 45], [10, 50], [10,60], [10,70],[10, 75], [15, 88], [20, 70], [20, 75], [20, 80], [30, 80]]

t = 1
t1 = np.linspace(0, 1, 100)
t2 = t - t1
fE_B = np.zeros_like(t1)
fE_A = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    fE_B = np.zeros_like(t1)
    for i in range(len(t1)):
        #print(t1 + t2)

        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]
        layupB_weakened = [{'mat':m2,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m2,  'ori':  -theta2,  'thi':t2[i]/2}]

        
        fE_B_ff = solve(layupB_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
        fE_B_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_B[i] = max(fE_B_ff,fE_B_iff)
        
    i = fE_B.argmin() # Optimal fE
    label = f"±{theta1}/±{theta2}"
    print("Min fE=", fE_B[i], "t1=", t1[i], "t2=", t2[i])
    plt.plot(t1, fE_B, label=label)
    
    
# Solve for layup A
theta1_A = 10
layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]
layupA_weakened =     [{'mat':m2,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m2,  'ori':  -theta1_A,  'thi':t/2}]


fE_A_iff = solve_old(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
fE_A_ff = solve_old(layupA_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
fE_A = (np.zeros_like(t1)+1)*max(fE_A_iff, fE_A_ff)

label_A = f"±{theta1_A}"
print("min_fE layupA", min(fE_A))
plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
plt.legend()

plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
plt.xlabel("$t_1$/$t$")
plt.ylabel("$f_E$")
plt.ylim([0,3])
plt.savefig(f'output/Case{casenr} - optimization.png')



In [ ]:
# Angle optimization for different thicknesses - IFF
# Case 3
casenr = 3
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [[10, 45], [10, 50], [10,60], [10,70],[10, 75], [15, 88], [20, 70], [20, 75], [20, 80], [30, 80]]

t = 1
t1 = np.linspace(0, 1, 100)
t2 = t - t1
fE_B = np.zeros_like(t1)
fE_A = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    fE_B = np.zeros_like(t1)
    for i in range(len(t1)):
        #print(t1 + t2)

        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]
        layupB_weakened = [{'mat':m2,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m2,  'ori':  -theta2,  'thi':t2[i]/2}]

        
        fE_B_ff = solve(layupB_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
        fE_B_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_B[i] = max(fE_B_ff,fE_B_iff)
        
    i = fE_B.argmin() # Optimal fE
    label = f"±{theta1}/±{theta2}"
    print("Min fE=", fE_B[i], "t1=", t1[i], "t2=", t2[i])
    plt.plot(t1, fE_B, label=label)
    
    
# Solve for layup A
theta1_A = 45
layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]
layupA_weakened =     [{'mat':m2,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m2,  'ori':  -theta1_A,  'thi':t/2}]


fE_A_iff = solve_old(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
fE_A_ff = solve_old(layupA_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_iff"]
fE_A = (np.zeros_like(t1)+1)*max(fE_A_iff, fE_A_ff)

label_A = f"±{theta1_A}"
print("min_fE layupA", min(fE_A))
plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
plt.legend()

plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
plt.xlabel("$t_1$/$t$")
plt.ylabel("$f_E$")
plt.ylim([min(fE_A)-2,min(fE_A)*2])
plt.savefig(f'output/Case{casenr} - optimization.png')



In [ ]:
# Angle optimization for different thicknesses - IFF
# Case 4
casenr = 4
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [[10, 45], [10, 50], [10,60], [10,70],[10, 75], [15, 88], [20, 70], [20, 75], [20, 80], [30, 80]]

t = 1
t1 = np.linspace(0, 1, 1000)
t2 = t - t1
fE_B = np.zeros_like(t1)
fE_A = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    fE_B = np.zeros_like(t1)
    for i in range(len(t1)):
        #print(t1 + t2)

        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]
        layupB_weakened = [{'mat':m2,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m2,  'ori':  -theta2,  'thi':t2[i]/2}]

        
        fE_B_ff = solve(layupB_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
        fE_B_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_B[i] = max(fE_B_ff,fE_B_iff)
        
    i = fE_B.argmin() # Optimal fE
    label = f"±{theta1}/±{theta2}"
    print("Min fE=", fE_B[i], "t1=", t1[i], "t2=", t2[i])
    plt.plot(t1, fE_B, label=label)
    
    
# Solve for layup A
theta1_A = 47
layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]
layupA_weakened =     [{'mat':m2,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m2,  'ori':  -theta1_A,  'thi':t/2}]


fE_A_iff = solve_old(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
fE_A_ff = solve_old(layupA_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_iff"]
fE_A = (np.zeros_like(t1)+1)*max(fE_A_iff, fE_A_ff)

label_A = f"±{theta1_A}"
print("min_fE layupA", min(fE_A))
plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
plt.legend()

plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
plt.xlabel("$t_1$/$t$")
plt.ylabel("$f_E$")
plt.ylim([min(fE_A)-2,min(fE_A)*2])
plt.savefig(f'output/Case{casenr} - optimization.png')

In [ ]:
# Angle optimization for different thicknesses - IFF
# Case 5
casenr = 5
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [[10, 45], [30,40], [10, 50], [10,60], [10,70],[10, 75], [15, 88], [20, 70], [20, 75], [20, 80], [30, 80]]

t = 1
t1 = np.linspace(0, 1, 100)
t2 = t - t1
fE_B = np.zeros_like(t1)
fE_A = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    fE_B = np.zeros_like(t1)
    for i in range(len(t1)):
        #print(t1 + t2)

        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]
        layupB_weakened = [{'mat':m2,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m2,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m2,  'ori':  -theta2,  'thi':t2[i]/2}]

        
        fE_B_ff = solve(layupB_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
        fE_B_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_B[i] = max(fE_B_ff,fE_B_iff)
        
    i = fE_B.argmin() # Optimal fE
    label = f"±{theta1}/±{theta2}"
    print("Min fE=", fE_B[i], "t1=", t1[i], "t2=", t2[i])
    plt.plot(t1, fE_B, label=label)
    
    
# Solve for layup A
theta1_A = 34
layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]
layupA_weakened =     [{'mat':m2,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m2,  'ori':  -theta1_A,  'thi':t/2}]


fE_A_iff = solve_old(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
fE_A_ff = solve_old(layupA_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_iff"]
fE_A = (np.zeros_like(t1)+1)*max(fE_A_iff, fE_A_ff)

label_A = f"±{theta1_A}"
print("min_fE layupA", min(fE_A))
plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
plt.legend()

plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
plt.xlabel("$t_1$/$t$")
plt.ylabel("$f_E$")
plt.ylim([min(fE_A)-2,min(fE_A)*2])
plt.savefig(f'output/Case{casenr} - optimization.png')



In [ ]:
# Angle optimization for different thicknesses - IFF
# Case 2
casenr = 2
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [ [10, 10], [45, 10]]

t = 1
t1 = np.linspace(0, 1, 100)
t2 = t - t1
fE_B = np.zeros_like(t1)
#fE_A = np.zeros_like(t1)
fE_AA = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    fE_B = np.zeros_like(t1)
    for i in range(len(t1)):
        #print(t1 + t2)

        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]
        
        layupA =     [{'mat':m,  'ori':  10,  'thi':t/2},
                     {'mat':m,  'ori':  -10,  'thi':t/2}]
        
        
#        layupA =     [{'mat':m,  'ori':  theta1,  'thi':t/2},
#                      {'mat':m,  'ori':  -theta1,  'thi':t/2}]

        fE_B[i] = solve_old(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_AA[i] = solve_old(layupA, R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
    #i_B = fE_B.argmin() # Optimal fE
    #i_A = fE_B.argmin() # Optimal fE
    label = f"±{theta1}/±{theta2}"
    #print("Min fE=", fE_B[i_B], "t1=", t1[i], "t2=", t2[i])
    
    plt.plot(t1, fE_B, label=label)
    plt.plot(t1, fE_AA, label=label)
    
    
# Solve for layup A
#theta1_A = 10
#layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
#              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]

#for i in range(len(t1)):
#    fE_A[i] = solve_old(layupA,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]

#label_A = f"±{theta1_A}"
#plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
#plt.legend()

#plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
#plt.xlabel("$t_1$")
#plt.ylabel("$f_E$ iff")
#plt.savefig(f'output/Case{casenr} - optimization.png')



In [ ]:
t = 1
t1 = 1
t2 = t-t1
theta1 = 10
theta2 = 10

layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1/2},
              {'mat':m,  'ori':  -theta1,  'thi':t1/2},
              {'mat':m,  'ori':  theta2,  'thi':t2/2},
              {'mat':m,  'ori':  -theta2,  'thi':t2/2}]

layupA =     [{'mat':m,  'ori':  10,  'thi':t/2},
             {'mat':m,  'ori':  -10,  'thi':t/2}]


#        layupA =     [{'mat':m,  'ori':  theta1,  'thi':t/2},
#                      {'mat':m,  'ori':  -theta1,  'thi':t/2}]

fE_B = solve_old(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
fE_A = solve_old(layupA, R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]

label = f"±{theta1}/±{theta2}"
print(f"{fE_A=}")
print(f"{fE_B=}")


In [ ]:
# Angle optimization for different thicknesses - FF
# Case 2 FF
casenr = 2
P,Fx,T = loadcase(casenr)
R = 75

set_plot_props()
thetaspace = [[10, 45], [10, 54], [50,55], [10,60], [10,70],[10, 75], [15, 88], [20, 70], [20, 75], [20, 80], [30, 80]]
#thetaspace = [[50,50]]
t = 1
t1 = np.linspace(0, 1, 100)
t2 = t - t1
fE_B = np.zeros_like(t1)
fE_A = np.zeros_like(t1)

for theta1, theta2 in thetaspace:
    for i in range(len(t1)):
        layupB =     [{'mat':m,  'ori':  theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  -theta1,  'thi':t1[i]/2},
                      {'mat':m,  'ori':  theta2,  'thi':t2[i]/2},
                      {'mat':m,  'ori':  -theta2,  'thi':t2[i]/2}]



        fE_B_ff = solve(layupB,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
        fE_B_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]
        fE_B[i] = max(fE_B_ff,fE_B_iff)
    i = fE_B.argmin() # Optimal fE_ff
    label = f"±{theta1}/±{theta2}"
    print("Min fE=", fE_B[i], "t1=", t1[i], "t2=", t2[i])
    plt.plot(t1, fE_B, label=label)
    
    
# Solve for layup A
theta1_A = 10

layupA =     [{'mat':m,  'ori':  theta1_A,  'thi':t/2},
              {'mat':m,  'ori':  -theta1_A,  'thi':t/2}]

for i in range(len(t1)):
    fE_A[i] = solve(layupA,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]

label_A = f"±{theta1_A}"
plt.plot(t1, fE_A, label=label_A, linewidth=2.5)
plt.legend()
plt.title(f"Case {casenr} optimization for seleced angle pairs t=1, $t_2$=t-$t_1$")
plt.xlabel("$t_1$")
plt.ylabel("$f_E$ iff")

#plt.savefig(f'output/Case{casenr} - optimization.png')


In [23]:
# Brute forcing search through all angles and 4 different thicknesses
casenr = 4
P,Fx,T = loadcase(casenr)
R = 75
set_plot_props()
theta1 = np.arange(10,88,1)
theta2 = np.arange(10,88,1)
t = 1
for t1 in [0.2, 0.4, 0.6, 0.8]:
    t2 = t - t1
    fE = [[0]*len(theta1)]*len(theta2)
    fE_min = 100
    th1_min, th2_min = -1, -1
    for i in range(len(theta1)):
        for j in range(len(theta2)):
            th1, th2 = theta1[i], theta2[j]
            layupB =     [{'mat':m,  'ori':  th1,  'thi':t1/2},
                          {'mat':m,  'ori':  -th1,  'thi':t1/2},
                          {'mat':m,  'ori':  th2,  'thi':t2/2},
                          {'mat':m,  'ori':  -th2,  'thi':t2/2}]

            layupB_weakened = [{'mat':m2,  'ori':  th1,  'thi':t1/2},
                               {'mat':m2,  'ori':  -th1,  'thi':t1/2},
                               {'mat':m2,  'ori':  th2,  'thi':t2/2},
                               {'mat':m2,  'ori':  -th2,  'thi':t2/2}]

            fE_ff = solve(layupB_weakened,R=75,P=P*3,Fx=Fx*3,T=T*3)["fE_ff"]
            fE_iff = solve(layupB,R=75,P=P*1.2,Fx=Fx*1.2,T=T*1.2)["fE_iff"]

            fE = max(fE_ff,fE_iff) 

            if fE < fE_min:
                th1_min, th2_min = theta1[i], theta2[j]
                fE_min = fE

    print("minmum fE", fE_min, f"given t1 = {t1}")
    print("Optimal angles", th1_min, th2_min)
    print("")

minmum fE 7.011810930492357 given t1 = 0.2
Optimal angles 47 47

minmum fE 7.011810930492357 given t1 = 0.4
Optimal angles 47 47

minmum fE 7.011810930492356 given t1 = 0.6
Optimal angles 47 47

minmum fE 7.011810930492356 given t1 = 0.8
Optimal angles 47 47



<Figure size 648x648 with 0 Axes>